## Create a Kubernetes (EKS) Cluster

Create an EKS cluster named “simple-jwt-api” and set up all the associated services:

>```eksctl create cluster --name simple-jwt-api```

The command above will take a few minutes to execute, and create an EKS cluster in your default region, as specified by your AWS CLI configuration, with one nodegroup containing two m5.large nodes. For more information on creating and managing cluster, refer to [https://eksctl.io/usage/creating-and-managing-clusters/]

You can go to the CloudFormation console to view progress. If you don’t see any progress, be sure that you are viewing clusters in the same region that they are being created. To confirm your region, you can run aws configure list command. Once the status is `‘CREATE_COMPLETE’` in your command line, check the health of your clusters nodes:

>```kubectl get nodes```

Remember, in case you wish to delete the cluster, you can do it either ways:

* CloudFormation console - select your stack and choose delete from the actions menu, or

* AWSCLI - delete using eksctl: eksctl delete cluster simple-jwt-api

This deletion step is crucial after you receive your project feedback.

## Create an IAM Role
The next steps are provided to quickly set up an IAM role to allow accessing your cluster. This is the role that CodeBuild will assume to administer the EKS cluster.

1. Get your AWS account id::

>```aws sts get-caller-identity --query Account --output text```

2. Create a role trust policy. To do this, first create a blank trust.json file, and add the following policy to it:

>```
{
 "Version": "2012-10-17",
 "Statement": [
     {
         "Effect": "Allow",
         "Principal": {
             "AWS": "arn:aws:iam::<ACCOUNT_ID>:root"
         },
         "Action": "sts:AssumeRole"
     }
 ]
}
```

Replace the `<ACCOUNT_ID>` with your actual account Id. This policy file defines the actions allowed by whosoever assumes the new Role.

3. Create a role, 'UdacityFlaskDeployCBKubectlRole', using the trust.json policy file:

>```aws iam create-role --role-name UdacityFlaskDeployCBKubectlRole --assume-role-policy-document file://trust.json --output text --query 'Role.Arn'```

4. Create another policy document, iam-role-policy.json, that allows additional actions "eks:Describe*" and "ssm:GetParameters" as:

>```
{
 "Version": "2012-10-17",
 "Statement":[{
     "Effect": "Allow",
     "Action": ["eks:Describe*", "ssm:GetParameters"],
     "Resource":"*"
 }]
}
```

5. Attach the iam-role-policy.json policy to the 'UdacityFlaskDeployCBKubectlRole' as:

>```aws iam put-role-policy --role-name UdacityFlaskDeployCBKubectlRole --policy-name eks-describe --policy-document file://iam-role-policy.json```


## Allowing the new role access to the cluster

1. Before you assign the new-role to the CodeBuild service (so that CodeBuild can also administer the cluster) you will have to add an entry of this new role into the 'aws-auth ConfigMap'. The aws-auth ConfigMap is used to grant role-based access control to your cluster. When your cluster is first created, the user who created it is given sole permission to administer it. Therefore, to grant any AWS service/user who will assume this role the ability to interact with your cluster, you must edit the 'aws-auth ConfigMap' within Kubernetes.

To do so, first, get the current configmap and save it to a file:

>```kubectl get -n kube-system configmap/aws-auth -o yaml > /tmp/aws-auth-patch.yml```

The file will be created at ```/System/Volumes/Data/private/tmp/aws-auth-patch.yml path```.

2. Open the aws-auth-patch.yml file using any editor, such as VS code editor:

>```code /System/Volumes/Data/private/tmp/aws-auth-patch.yml```

and add the following group in the data → mapRoles section of this file:

>```
mapRoles: |
 - groups:
   - system:masters
   rolearn: arn:aws:iam::<ACCOUNT_ID>:role/UdacityFlaskDeployCBKubectlRole
   username: build      
```

Don't forget to replace the `<ACCOUNT_ID>` with your AWS account Id. See a snapshot below.

3. Update your cluster's configmap:

>```kubectl patch configmap/aws-auth -n kube-system --patch "$(cat /tmp/aws-auth-patch.yml)"```

The command above must show you `configmap/aws-auth patched` as response. Otherwise, in case of the following error:

>```Error from server (Conflict): Operation cannot be fulfilled on configmaps "aws-auth": the object has been modified; please apply your changes to the latest version and try again```

re-run these three step beginning from the `kubectl get` command.

## Deployment to Kubernetes using CodePipeline and CodeBuild
You should now be ready to deploy your application using CodePipeline and CodeBuild.

You will now create a pipeline that watches your Github. When changes are checked in, it will build a new image and deploy it to your cluster. When you created your cluster, `eksctl` automatically created a CloudFormation stack, now we will use a CloudFormation template to create a stack for a CodePipeline using the AWS console. Follow the steps below to complete your project.

1. Generate a GitHub Access Token

A Github access token will allow CodePipeline to monitor when a repo is changed. A token can be generated [here](https://github.com/settings/tokens/). You should generate the token with full control of private repositories, as shown in the image below. Be sure to save the token somewhere that is secure.


Once you create a personal access token, you can share this with any service (such as AWS, or CLI) to allow accessing the repositories under your Github account.

2. Create the CodePipeline Pipeline using CloudFormation Template

a. Modify CloudFormation template - There is a file named **ci-cd-codepipeline.cfn.yml** provided in your starter repo. This is the template file that you will use to create your CodePipeline pipeline and "build project" in CodeBuild. Open this file, and go to the 'Parameters' section. These are parameters that will accept values when you create a stack. Fill in the value for the following parameters:

| Parameter | Field | Possible Value |
| :----: | :----: | :----: |
| EksClusterName    |  Default |    simple-jwt-api (Name of the EKS cluster you created) |
| GitSourceRepo |   Default |   FSND-Deploy-Flask-App-to-Kubernetes-Using-EKS (Github repo name) |
| GitBranch |   Default |   master (Or any other you want to link to the Pipeline) |
| GitHubUser    |  Default  |    Your Github username |
| KubectlRoleName   |   Default |   UdacityFlaskDeployCBKubectlRole (We created this role earlier) |


b. Create a stack for CodePipeline using the CloudFormation template (ci-cd-codepipeline.cfn.yml) file. Go to the CloudFormation service in the AWS console. Press the 'Create Stack' button. It will make you go through the following three steps -

* Step 1 - Specify template - Choose the 'Upload a template file ' option and upload the template file 'ci-cd-codepipeline.cfn.yml'

* Step 2 - Specify stack details - Press 'Next'. Give the stack a name, fill in your GitHub login, and the Github access token generated in the previous step. Make sure that the cluster name matches the one you have created, and the 'kubectl IAM role' matches the role you created above, and the repository matches the name of your forked repo.

* Step 3 - Configure stack options - Leave default, and create the stack. You can check its status in the [CloudFormation console].

It will take some time (5-15 mins) to create the stack. After the successful creation of the stack, you can see the **CodeBuild** and **CodePipeline** project gets automatically created for you. Although, the Cloudformation template is creating a few more resources, such as an S3 bucket, a Lambda function, and others.


3. Set a Secret using AWS Parameter Store
We need a way to pass your JWT secret to the app in kubernetes securly. You will be using AWS Parameter Store to do this. First, add the following to the end of the `buildspec.yml` file (be careful about the indentation):

>```
   env:
     parameter-store:         
       JWT_SECRET: 
```

This lets CodeBuild know to set an environment variable based on a value in the parameter-store. Put secret into AWS Parameter Store

>```aws ssm put-parameter --name JWT_SECRET --overwrite --value "YourJWTSecret" --type SecureString```

Once you submit your project and receive the reviews, you can consider deleting the variable from parameter-store using:

>```aws ssm delete-parameter --name JWT_SECRET```


4. Make use of buildspec.yml for Build using CodeBuild

The file buildspec.yml file specifies the different phases of a build, such as an install, pre-build, build, and post-build. Each phase has a set of commands to be automatically executed by CodeBuild. When you trigger a build in the CodeBuild, you can see each command being executed in the CodeBuild log console.

>```
In the previous step, the CloudFormation template file ci-cd-codepipeline.cfn.yml would automatically create a "build project" in CodeBuild. You just have to start the build process by clicking on the “Start build” button. The build process will make use of buildspec.yml file automatically.
```

![start the build](C:\Users\musta\OneDrive\Documents\Programming\FSND-Deploy-Flask-App-to-Kubernetes-Using-EKS\images\start-build.png)

![build in progress](C:\Users\musta\OneDrive\Documents\Programming\FSND-Deploy-Flask-App-to-Kubernetes-Using-EKS\images\build-progress.png)

### Successfully Working Pipeline

1. Check the pipeline works. Once the stack is successfully created, commit a change to the master branch of your github repo, as:
>```
git remote -v
git add . --all
git commit -m “my comment“
git push```

Then, in the aws console go to the `CodePipeline UI`. You should see that the build is running.

2. To test your api endpoints, get the external IP for your service:

>```kubectl get services simple-jwt-api -o wide```

Now use the external IP url to test the app:

>```
export TOKEN=`curl -d '{"email":"<EMAIL>","password":"<PASSWORD>"}' -H "Content-Type: application/json" -X POST <EXTERNAL-IP URL>/auth  | jq -r '.token'`
```

>```curl --request GET '<EXTERNAL-IP URL>/contents' -H "Authorization: Bearer ${TOKEN}" | jq```

3. Save the external IP from above to provide to the reviewer when you will submit your project. Go to the next page for details about the project submission process.


Adding Tests to the Build
The final part of this project involves adding tests to your deployment. You can follow the steps below to accomplish this.

## Add running tests as part of the build.

To require the unit tests to pass before our build will deploy new code to your cluster, you will add the tests to the build stage. Remember you installed the requirements and ran the unit tests locally at the beginning of this project. You will add the same commands to the buildspec.yml:

1. Open buildspec.yml
In the prebuild section, add a line to install the requirements and a line to run the tests. You may need to refer to 'pip' as 'pip3' and 'python' as 'python3', as:

>```
pre_build:
commands:
  - pip3 install -r requirements.txt 
  - python -m pytest test_main.py
```

save the file

2. You can check the tests prevent a bad deployment by breaking the tests on purpose:

* Open the `test_main.py` file
* Add `assert False` to any of the tests
* Commit your code and push it to Github
* Check that the build fails in CodePipeline